In [23]:
%load_ext autoreload
%autoreload 2

In [24]:
from src.dataset.Loader.main_loader import define_path
from src.dataset.trans.data import TransDataset, extract_pred_sequence
from src.dataset.trans.jaad_trans import JaadTransDataset
from src.dataset.intention.jaad_dataset import build_pedb_dataset_jaad, subsample_and_balance
import cv2
import numpy as np
import glob
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt

from src.dataset.loader import PaddedSequenceDataset, IntentionSequenceDataset
from src.transform.preprocess import ImageTransform, Compose, CropBox
import torchvision
import torch

In [25]:
anns_paths, image_dir = define_path(use_jaad=True, use_pie=False, use_titan=False)
anns_paths_val, image_dir_val = define_path(use_jaad=True, use_pie=False, use_titan=False)

In [32]:
from dataclasses import dataclass

@dataclass
class Args:
    encoder_type: str = 'CC'
    encoder_pretrained: bool = False
    epochs: int = 1
    lr: float = 0.001
    wd: float = 0.0
    batch_size: int = 4
    max_frames: int = 10
    pred: int = 10
    output: str = None
    fps: int = 5
    seed: int = 99
    jitter_ratio: float = -1.0
    mobilenetsmall: bool = False
    mobilenetbig: bool = False
    num_workers: int = 4

args = Args(num_workers=0)
max_frames = 10

In [35]:
image_set = "test"


intent_sequences = build_pedb_dataset_jaad(anns_paths["JAAD"]["anns"], anns_paths["JAAD"]["split"], image_set=image_set, fps=args.fps, prediction_frames=args.pred, verbose=True)
balance = False if image_set == "test" else True
intent_sequences_cropped = subsample_and_balance(intent_sequences, max_frames=args.max_frames, seed=args.seed, balance=balance)

jitter_ratio = None if args.jitter_ratio < 0 else args.jitter_ratio
crop_preprocess = CropBox(size=224, padding_mode='pad_resize', jitter_ratio=jitter_ratio)
if image_set == 'train':
    TRANSFORM = Compose([
        crop_preprocess,
        ImageTransform(torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1))
        ])
else:
    TRANSFORM = crop_preprocess

ds = IntentionSequenceDataset(intent_sequences_cropped[:500], image_dir=image_dir, hflip_p = 0.5, preprocess=TRANSFORM)

----------------------------------------------------------------
JAAD:
Total number of crosses: 4002
Total number of non-crosses: 2196
Filtered samples: 17
Total number of samples before and after balancing: 3794, 3794


In [28]:
from train_hybrid import eval_model
from src.model.models import build_encoder_res18, DecoderRNN_IMBS
from src.early_stopping import load_from_checkpoint

CP_PATH = '/Users/arinaruck/Desktop/courses/CIVIL459-PedestrianIntensionDetection/checkpoints/silver-sweep-5/Decoder_IMBS_lr0.001_wd0.0001_JAAD_mf10_pred10_bs16_202305241656.pt'

In [30]:
device = torch.device('cpu')


encoder_res18 = build_encoder_res18(args)
# freeze CNN-encoder during training
encoder_res18.freeze_backbone()

decoder_lstm = DecoderRNN_IMBS(CNN_embeded_size=256, h_RNN_0=256, h_RNN_1=64, h_RNN_2=16,
                                h_FC0_dim=128, h_FC1_dim=64, h_FC2_dim=86, drop_p=0.2).to(device)

model = {'encoder': encoder_res18, 'decoder': decoder_lstm}
load_from_checkpoint(model, CP_PATH)

Using resnet18 cnn encoder!!


In [36]:
#test_loader = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=False, num_workers=args.num_workers, pin_memory=True)
#eval_model(test_loader, model, device)